In [1]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import data as dt
import functions as fn
from collections import Counter

In [2]:
BTCUSDT15m = dt.BTCUSDT15m
BTCUSDT15m.set_index("Open Time", inplace=True)

In [3]:
# SRSI
BTCUSDT15m = pd.concat([BTCUSDT15m, pta.stochrsi(BTCUSDT15m.Close, 14)], axis = 1)
BTCUSDT15m.rename(columns = {'STOCHRSIk_14_14_3_3':'K', 'STOCHRSId_14_14_3_3':'D'}, inplace = True)

In [4]:
# EMA
for i in (8, 14, 50):
    BTCUSDT15m['EMA_'+str(i)] = pta.ema(BTCUSDT15m.Close, i)

In [5]:
# ATR
BTCUSDT15m['ATR'] =pta.atr(BTCUSDT15m.High, BTCUSDT15m.Low, BTCUSDT15m.Close)

In [6]:
BTCUSDT15m.dropna(inplace=True)

In [7]:
def check_cross(df):
    series = df['K'] > df['D']
    return series.diff()

In [8]:
BTCUSDT15m['KD_Cross'] = check_cross(BTCUSDT15m)
BTCUSDT15m.dropna(inplace=True)

In [9]:
BTCUSDT15m['TP'] = BTCUSDT15m.Close+(BTCUSDT15m.ATR*2)

In [10]:
BTCUSDT15m['SL'] = BTCUSDT15m.Close-(BTCUSDT15m.ATR*3)

In [11]:
BTCUSDT15m.head(2)

,Open,High,Low,Close,Volume,K,D,EMA_8,EMA_14,EMA_50,ATR,KD_Cross,TP,SL
Open Time,,,,,,,,,,,,,,
2018-01-01 12:30:00,13146.81,13245.55,13146.28,13200.01,66.739212,38.382290,35.686151,13253.118039,13329.591503,13478.741024,151.754605,False,13503.519210,12744.746185
2018-01-01 12:45:00,13200.01,13225.00,13117.72,13172.42,54.989819,36.260385,37.124182,13235.185141,13308.635303,13466.728434,148.503614,True,13469.427229,12726.909157


In [12]:
BTCUSDT15m['Buy_signal'] = np.where((BTCUSDT15m.KD_Cross) &
                                     (BTCUSDT15m.Close > BTCUSDT15m.EMA_8) &
                                     (BTCUSDT15m.EMA_8 > BTCUSDT15m.EMA_14) &
                                     (BTCUSDT15m.EMA_14 > BTCUSDT15m.EMA_50), 1, 0)

In [13]:
sells = []
outcome = []
for i in range(len(BTCUSDT15m)):
    if BTCUSDT15m.Buy_signal.iloc[i]:
        k = 1
        SL = BTCUSDT15m.SL.iloc[i] # Stop Loss
        TP = BTCUSDT15m.TP.iloc[i] # Take Profit
        position = True
        while position:
            l_close = BTCUSDT15m.Close.iloc[i+k]
            if l_close >= TP:
                sells.append(BTCUSDT15m.iloc[i+k].name)
                outcome.append('TP')
                position = False
            elif l_close <= SL:
                sells.append(BTCUSDT15m.iloc[i+k].name)
                outcome.append('SL')
                position = False
            k+=1

In [14]:
BTCUSDT15m.loc[sells, 'Sell_signal'] = 1

In [15]:
BTCUSDT15m.loc[sells, 'Outcome'] = outcome

In [16]:
BTCUSDT15m.Sell_signal = BTCUSDT15m.Sell_signal.fillna(0).astype(int)

In [17]:
mask = BTCUSDT15m[(BTCUSDT15m.Buy_signal == 1) | (BTCUSDT15m.Sell_signal == 1)]

In [18]:
mask2 = mask[(mask.Buy_signal.diff() == 1) | (mask.Sell_signal.diff() == 1)]

In [19]:
RES = mask2.Outcome.value_counts()
RES

TP    2090
SL    1311
Name: Outcome, dtype: int64

In [20]:
win_rate = RES[0]/(RES[0]+RES[1])
win_rate

0.6145251396648045

In [21]:
loss_rate = RES[1]/(RES[0]+RES[1])
loss_rate

0.3854748603351955

In [30]:
train = BTCUSDT15m['2018-01-01 00:00:00': '2018-02-01 00:00:00']

In [31]:
fig_001 = go.Figure(
    data=[
        go.Candlestick(
            x=train.index,
            open=train["Open"],
            high=train["High"],
            low=train["Low"],
            close=train["Close"]
        )])
#         go.Scatter(x=t1.index, y=t1['K'], line=dict(color='blue', width=1.5),mode="lines", name="K", yaxis="y2"),
#         go.Scatter(x=t1.index, y=t1['D'], line=dict(color='red', width=1.5),mode="lines", name="D", yaxis="y2"),
#         go.Scatter(x=t1.index, y=t1.VWAP, line=dict(color='blue', width=1.5), name="VWAP"),
#         go.Scatter(x=pivs.index, y=pivs.p, line=dict(color='red', width=1.5), name="P"),
#         go.Scatter(x=pivs.index, y=pivs.s1, line=dict(color='orange', width=1), name="S1"),
#         go.Scatter(x=pivs.index, y=pivs.s2, line=dict(color='orange', width=1), name="S2"),
#         go.Scatter(x=pivs.index, y=pivs.r1, line=dict(color='orange', width=1), name="R1"),
#         go.Scatter(x=pivs.index, y=pivs.r2, line=dict(color='orange', width=1), name="R2")]
#         ).update_layout(
#         yaxis_domain=[0.3, 1],
#         yaxis2={"domain": [0, 0.20]},
#         xaxis_rangeslider_visible=False,
#         showlegend=False,
#  )

# fig_001.add_scatter(x=t1.index, y=t1['pointpos'], mode="markers",
#                 marker=dict(size=7, color="MediumPurple"),
#                 name="Signal")

fig_001.show()

# VWAP SIGNAL

In [23]:
%%time
BTCUSDT15m['VWAP'] = fn.vwap(BTCUSDT15m.High, BTCUSDT15m.Low, BTCUSDT15m.Close, BTCUSDT15m.Volume)

CPU times: total: 62.5 ms
Wall time: 39.9 ms


In [24]:
BTCUSDT15m['VWAP_buy_signal'] = BTCUSDT15m['Close'] > BTCUSDT15m['VWAP']
BTCUSDT15m['VWAP_buy_signal'] = BTCUSDT15m['VWAP_buy_signal'].rolling(3).sum() >= 3
BTCUSDT15m['VWAP_buy_signal'] = BTCUSDT15m['VWAP_buy_signal'].astype(int)

# PIVOTS SIGNAL

In [25]:
pivots = fn.pivots(BTCUSDT15m['Open'], BTCUSDT15m['High'], BTCUSDT15m['Low'], BTCUSDT15m['Close'], anchor=15, method='fibonacci')

# SRSI SIGNAL

In [26]:
%%time
BTCUSDT15m = pd.concat([BTCUSDT15m, pta.stochrsi(BTCUSDT15m.Close, 14)], axis = 1)

CPU times: total: 46.9 ms
Wall time: 52.8 ms


In [27]:
BTCUSDT15m.rename(columns = {'STOCHRSIk_14_14_3_3':'K', 'STOCHRSId_14_14_3_3':'D'}, inplace = True)

In [28]:
BTCUSDT15m.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'K', 'D', 'EMA_8', 'EMA_14',
       'EMA_50', 'ATR', 'KD_Cross', 'TP', 'SL', 'Buy_signal', 'Sell_signal',
       'Outcome', 'VWAP', 'VWAP_buy_signal', 'K', 'D'],
      dtype='object')

In [29]:
BTCUSDT15m['SRSI_buy'] = BTCUSDT15m['K'] > BTCUSDT15m['D']
BTCUSDT15m['SRSI_sell'] = BTCUSDT15m['K'] < BTCUSDT15m['D']

BTCUSDT15m['SRSI_buy_signal'] = BTCUSDT15m['SRSI_buy'].rolling(2).sum() >= 2
BTCUSDT15m['SRSI_buy_signal'] = BTCUSDT15m['SRSI_buy_signal'].astype(int)
BTCUSDT15m['SRSI_sell_signal'] = BTCUSDT15m['SRSI_sell'].rolling(3).sum() >= 3

ValueError: Can only compare identically-labeled DataFrame objects

In [ ]:
BTCUSDT15m['SRSI_sell_signal'] = BTCUSDT15m['SRSI_sell_signal'].map({True: 2 , False: 0})

In [ ]:
BTCUSDT15m['SRSI_signal'] = BTCUSDT15m['SRSI_buy_signal']+BTCUSDT15m['SRSI_sell_signal']

In [ ]:
BTCUSDT15m[['VWAP_buy_signal', 'SRSI_signal']]

# TOTAL SIGNAL

In [ ]:
BTCUSDT15m['buy'] = [1 if BTCUSDT15m['VWAP_buy_signal'][i] == 1 and BTCUSDT15m['SRSI_signal'][i] == 1 else 0 for i in range(len(BTCUSDT15m))]

In [ ]:
Counter(BTCUSDT15m['buy'])

In [ ]:
BTCUSDT15m['sell'] = [2 if BTCUSDT15m['SRSI_signal'][i] == 2 else 0 for i in range(len(BTCUSDT15m))]

In [ ]:
Counter(BTCUSDT15m['sell'])

In [ ]:
BTCUSDT15m['position'] = BTCUSDT15m['sell']+BTCUSDT15m['buy']

In [ ]:
Counter(BTCUSDT15m['position'])

In [ ]:
def pointpos(x):
    if x['position']== 1:
        return x['High']+1e-3
    elif x['position']== 2:
        return x['Low']-1e-3
    else:
        return np.nan

BTCUSDT15m['pointpos'] = BTCUSDT15m.apply(lambda row: pointpos(row), axis=1)

In [ ]:
t1 = BTCUSDT15m['2021-03-18 19:00:00': '2021-03-21 00:00:00']
train = BTCUSDT15m['2018-01-01 00:00:00': '2019-01-01 00:00:00']
test = BTCUSDT15m['2019-02-01 00:00:00': '2020-02-01 00:00:00']

In [ ]:
%%time
pivs = fn.pivots(t1['Open'], t1['High'], t1['Low'], t1['Close'], anchor=15, method='fibonacci')

In [ ]:
fig_001 = go.Figure(
    data=[
        go.Candlestick(
            x=t1.index,
            open=t1["Open"],
            high=t1["High"],
            low=t1["Low"],
            close=t1["Close"],
            name="OHLC",
        ),
        go.Scatter(x=t1.index, y=t1['K'], line=dict(color='blue', width=1.5),mode="lines", name="K", yaxis="y2"),
        go.Scatter(x=t1.index, y=t1['D'], line=dict(color='red', width=1.5),mode="lines", name="D", yaxis="y2"),
        go.Scatter(x=t1.index, y=t1.VWAP, line=dict(color='blue', width=1.5), name="VWAP"),
        go.Scatter(x=pivs.index, y=pivs.p, line=dict(color='red', width=1.5), name="P"),
        go.Scatter(x=pivs.index, y=pivs.s1, line=dict(color='orange', width=1), name="S1"),
        go.Scatter(x=pivs.index, y=pivs.s2, line=dict(color='orange', width=1), name="S2"),
        go.Scatter(x=pivs.index, y=pivs.r1, line=dict(color='orange', width=1), name="R1"),
        go.Scatter(x=pivs.index, y=pivs.r2, line=dict(color='orange', width=1), name="R2")]
        ).update_layout(
        yaxis_domain=[0.3, 1],
        yaxis2={"domain": [0, 0.20]},
        xaxis_rangeslider_visible=False,
        showlegend=False,
 )

fig_001.add_scatter(x=t1.index, y=t1['pointpos'], mode="markers",
                marker=dict(size=7, color="MediumPurple"),
                name="Signal")

fig_001

![](files/strategy.png)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=972d23a0-f342-484b-9096-244a1ebf7fc5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>